# Датасет собирал из отзывов стим, ниже приведена информация о нем

In [1]:
appid = 1086940 # Baldurs Gate 3 (игра по которой собирал отзывы)
import pandas as pd
import json
import requests
from time import sleep

In [2]:
def get_reviews(appid, max_reviews=5000):
    reviews = []
    cursor_positive = '*'
    cursor_negative = '*'
    has_more_positive = True
    has_more_negative = True

    print("Собираем отрицательные отзывы...")
    # Сначала собираем все отрицательные отзывы
    while has_more_negative and len(reviews) < max_reviews:
        params = {
            'json': 1,
            'filter': 'all',
            'language': 'russian',
            'day_range': 9223372036854775807,
            'review_type': 'negative',
            'purchase_type': 'all',
            'cursor': cursor_negative,
            'num_per_page': min(100, max_reviews - len(reviews))
        }

        url = f"https://store.steampowered.com/appreviews/{appid}"
        response = requests.get(url, params=params)

        if response.status_code != 200:
            print(f"Ошибка: {response.status_code} для отрицательных отзывов")
            has_more_negative = False
            continue

        data = response.json()
        new_reviews = data['reviews']

        if not new_reviews:
            has_more_negative = False
        else:
            reviews.extend(new_reviews)
            cursor_negative = data['cursor']

        print(f"Собрано отрицательных отзывов: {len(reviews)}")
        sleep(0.3)

    negative_count = len(reviews)
    reviews = remove_duplicates(reviews)
    print(f"Всего собрано отрицательных отзывов: {negative_count}")
    # Если набрали достаточно отрицательных, возвращаем результат
    if len(reviews) >= max_reviews:
        print('Отрицательных отзывов достоточно')
        return reviews[:max_reviews]

    print("Добираем положительными")
    # Затем добираем положительными отзывами до нужного количества
    while has_more_positive and len(reviews) < max_reviews:
        params = {
            'json': 1,
            'filter': 'all',
            'language': 'russian',
            'day_range': 365,
            'review_type': 'positive',
            'purchase_type': 'all',
            'cursor': cursor_positive,
            'num_per_page': min(100, max_reviews - len(reviews))
        }

        url = f"https://store.steampowered.com/appreviews/{appid}"
        response = requests.get(url, params=params)

        if response.status_code != 200:
            print(f"Ошибка: {response.status_code} для положительных отзывов")
            has_more_positive = False
            continue

        data = response.json()
        new_reviews = data['reviews']

        if not new_reviews:
            has_more_positive = False
        else:
            reviews.extend(new_reviews)
            cursor_positive = data['cursor']

        print(f"Собрано отзывов: {len(reviews)} (отрицательных: {negative_count}, положительных: {len(reviews) - negative_count})")
        sleep(0.3)

    reviews = remove_duplicates(reviews)
    print(f"Итог: {len(reviews)} отзывов (отрицательных: {negative_count}, положительных: {len(reviews) - negative_count})")
    return reviews[:max_reviews]

def remove_duplicates(reviews):
    seen = set()
    unique_reviews = []
    for review in reviews:
        review_id = review['recommendationid']
        if review_id not in seen:
            seen.add(review_id)
            unique_reviews.append(review)
    return unique_reviews

In [3]:
reviews_data = get_reviews(appid, 8000)

Собираем отрицательные отзывы...
Собрано отрицательных отзывов: 99
Собрано отрицательных отзывов: 199
Собрано отрицательных отзывов: 299
Собрано отрицательных отзывов: 399
Собрано отрицательных отзывов: 499
Собрано отрицательных отзывов: 599
Собрано отрицательных отзывов: 699
Собрано отрицательных отзывов: 799
Собрано отрицательных отзывов: 899
Собрано отрицательных отзывов: 999
Собрано отрицательных отзывов: 1099
Собрано отрицательных отзывов: 1199
Собрано отрицательных отзывов: 1299
Собрано отрицательных отзывов: 1399
Собрано отрицательных отзывов: 1499
Собрано отрицательных отзывов: 1599
Собрано отрицательных отзывов: 1699
Собрано отрицательных отзывов: 1799
Собрано отрицательных отзывов: 1899
Собрано отрицательных отзывов: 1999
Собрано отрицательных отзывов: 2099
Собрано отрицательных отзывов: 2199
Собрано отрицательных отзывов: 2299
Собрано отрицательных отзывов: 2399
Собрано отрицательных отзывов: 2499
Собрано отрицательных отзывов: 2599
Собрано отрицательных отзывов: 2699
Собран

In [4]:
df = pd.DataFrame(reviews_data)
# df = pd.read_csv("steam_reviews_1086940.csv")

df.to_csv(f'steam_reviews_{appid}.csv', index=False, encoding='utf-8-sig')
print(f"Данные сохранены в steam_reviews_{appid}.csv")

Данные сохранены в steam_reviews_1086940.csv


In [5]:
df = pd.read_csv("steam_reviews_1086940.csv")
print(f"Всего собрано: {len(df)} отзывов")
print(df.info())

Всего собрано: 8000 отзывов
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   recommendationid             8000 non-null   int64  
 1   author                       8000 non-null   object 
 2   language                     8000 non-null   object 
 3   review                       7997 non-null   object 
 4   timestamp_created            8000 non-null   int64  
 5   timestamp_updated            8000 non-null   int64  
 6   voted_up                     8000 non-null   bool   
 7   votes_up                     8000 non-null   int64  
 8   votes_funny                  8000 non-null   int64  
 9   weighted_vote_score          8000 non-null   float64
 10  comment_count                8000 non-null   int64  
 11  steam_purchase               8000 non-null   bool   
 12  received_for_free            8000 non-null   boo

In [6]:
df['voted_up'].value_counts() # Положительный или отрицательный отзыв

,count
voted_up,
True,5225
False,2775


In [7]:
print(f"Размер датафрейма: {len(df)}")
print(f"Дубликатов: {df.duplicated(subset=['recommendationid']).sum()}")

Размер датафрейма: 8000
Дубликатов: 0


In [12]:
df_shuffled = df.sample(frac=1)

In [14]:
df_shuffled.head(10)

,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,primarily_steam_deck
3643,195474377,"{'steamid': '76561199485259582', 'num_games_ow...",russian,"Отличная игра с прекрасным сюжетом, интересным...",1748008570,1748008570,True,0,0,0.500000,0,True,False,False,False
4622,192798141,"{'steamid': '76561198070094696', 'num_games_ow...",russian,Сперва мне было противно играть в эту гендерно...,1744724574,1744724574,True,0,2,0.453446,0,True,False,False,False
573,167450676,"{'steamid': '76561198451680691', 'num_games_ow...",russian,"Чем дольше я играл в эту игру, тем больше осоз...",1718490423,1718490423,False,4,1,0.516000,6,True,False,False,False
3070,201764206,"{'steamid': '76561199259378458', 'num_games_ow...",russian,Игра настолька шедевральна что я хотел бы стер...,1754735965,1754735965,True,1,0,0.521739,1,False,False,False,False
2025,177085613,"{'steamid': '76561198069916832', 'num_games_ow...",russian,"Скажу сразу, я не хотел писать обзор пока не д...",1728982763,1728982763,False,10,0,0.487834,2,False,False,False,False
7265,181074894,"{'steamid': '76561198005968234', 'num_games_ow...",russian,"Качественная РПГ, жаль что таких уже практичес...",1732979785,1732979785,True,0,0,0.500000,0,False,False,False,False
1283,155682568,"{'steamid': '76561198006149831', 'num_games_ow...",russian,А вот так вот. Не рекомендую.\nBG3 это проект ...,1704886829,1706031578,False,10,0,0.434185,0,True,False,False,False
7901,179421653,"{'steamid': '76561199159280374', 'num_games_ow...",russian,Будь это кальмар или медведь в этой игре можно...,1732102425,1732102425,True,0,0,0.500000,0,False,False,False,False
7112,190379830,"{'steamid': '76561198064534562', 'num_games_ow...",russian,"We played in two, I recommend it to everyone w...",1742142957,1742142957,True,0,0,0.500000,0,True,False,False,False
3828,191820510,"{'steamid': '76561199434180464', 'num_games_ow...",russian,"Хорошая игра, Внимание спойлеры!!!!!\r\n\r\n\r...",1743618655,1743618655,True,0,0,0.500000,0,True,False,False,False
